In [6]:
import requests
import os
import time

serviceKey = "ePSdJxdLS6tqJrxStuufUYpC2pFIX0nPUGq94RE45W5ipNMft6nTH94Hj63v5fwjjSYjKTINLuo0uaHOAa6rqg=="
base_path = "./image"

# 1. 위치 기반으로 관광지 리스트 가져오기
def get_busan_locations(mapX, mapY):
    list_url = "https://apis.data.go.kr/B551011/KorService2/locationBasedList2"
    params = {
        'serviceKey': serviceKey,
        'numOfRows': '30',
        'MobileOS': 'ETC',
        'MobileApp': 'AppTest',
        '_type': 'json',
        'mapX': mapX,
        'mapY': mapY,
        'radius': '5000',
        'contentTypeId': '12',
        'arrange': 'O'
    }
    
    try:
        res = requests.get(list_url, params=params)
        if res.status_code == 200:
            data = res.json()
            items = data.get('response', {}).get('body', {}).get('items', {}).get('item', [])
            return [items] if isinstance(items, dict) else items
    except:
        return []

# 2. 특정 관광지의 상세 이미지들(갤러리) 가져오기
def get_detail_images(contentId, title):
    img_url = "https://apis.data.go.kr/B551011/KorService2/detailImage2"
    params = {
        'serviceKey': serviceKey,
        'MobileOS': 'ETC',
        'MobileApp': 'AppTest',
        '_type': 'json',
        'contentId': contentId,
        'imageMsType': '1'  # 1: 갤러리 이미지
    }
    
    try:
        res = requests.get(img_url, params=params)
        if res.status_code == 200:
            data = res.json()
            images = data.get('response', {}).get('body', {}).get('items', {}).get('item', [])
            img_list = [images] if isinstance(images, dict) else images
            
            # 저장 폴더 생성 (장소 이름)
            save_dir = os.path.join(base_path, title.replace("/", "_"))
            if not os.path.exists(save_dir): os.makedirs(save_dir)
            
            for i, img in enumerate(img_list):
                url = img.get('originimgurl')
                if url:
                    img_data = requests.get(url).content
                    with open(os.path.join(save_dir, f"{contentId}_{i}.jpg"), 'wb') as f:
                        f.write(img_data)
                    print(f"✅ {title} 저장 중... ({i+1}번째)")
    except Exception as e:
        print(f"❌ {title} 이미지 로드 실패: {e}")

# 실행: 해운대 좌표 기준
locations = get_busan_locations('129.1585', '35.1587')
for loc in locations:
    get_detail_images(loc['contentid'], loc['title'])
    time.sleep(0.5) # 과부하 방지

In [9]:
import requests
import os
import time

# API 설정
serviceKey = "ePSdJxdLS6tqJrxStuufUYpC2pFIX0nPUGq94RE45W5ipNMft6nTH94Hj63v5fwjjSYjKTINLuo0uaHOAa6rqg=="
base_path = "./image"

if not os.path.exists(base_path):
    os.makedirs(base_path)

def get_busan_data(mapX, mapY):
    # 1. 위치 기반 목록 조회 (성공했던 URL 형식)
    list_url = "https://apis.data.go.kr/B551011/KorService2/locationBasedList2"
    params = {
        'serviceKey': serviceKey,
        'numOfRows': '50',
        'pageNo': '1',
        'MobileOS': 'ETC',
        'MobileApp': 'AppTest',
        '_type': 'json',
        'arrange': 'O',
        'mapX': mapX,
        'mapY': mapY,
        'radius': '5000',
        'contentTypeId': '12'
    }
    
    try:
        print(f"\n🚀 좌표 ({mapX}, {mapY}) 주변 관광지 검색을 시작합니다...")
        res = requests.get(list_url, params=params, timeout=15)
        
        if res.status_code == 200:
            data = res.json()
            items_container = data.get('response', {}).get('body', {}).get('items')
            
            if not items_container or items_container == "":
                print("❌ 해당 구역에 검색 결과가 없습니다.")
                return

            items = items_container.get('item', [])
            if isinstance(items, dict): items = [items]

            print(f"✅ 총 {len(items)}개의 장소를 발견했습니다.")
            print("-" * 50)

            for item in items:
                contentId = item.get('contentid')
                title = item.get('title') # 관광지 명칭
                
                # 화면에 명칭 출력
                print(f"📍 현재 수집 중인 장소: [{title}] (ID: {contentId})")
                
                # 2. 상세 이미지 조회 및 저장 호출
                fetch_and_save_detail_images(contentId, title)
                time.sleep(0.3) # 서버 과부하 방지
        else:
            print(f"❌ 목록 조회 실패 (에러코드: {res.status_code})")

    except Exception as e:
        print(f"❗ 오류 발생: {e}")

def fetch_and_save_detail_images(contentId, title):
    # 상세 이미지 조회 URL
    detail_url = "https://apis.data.go.kr/B551011/KorService2/detailImage2"
    params = {
        'serviceKey': serviceKey,
        'MobileOS': 'ETC',
        'MobileApp': 'AppTest',
        '_type': 'json',
        'contentId': contentId,
        'imageMsType': '1'
    }
    
    try:
        res = requests.get(detail_url, params=params, timeout=10)
        if res.status_code == 200:
            data = res.json()
            img_items = data.get('response', {}).get('body', {}).get('items')
            
            if not img_items or img_items == "":
                print(f"   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.")
                return

            images = img_items.get('item', [])
            if isinstance(images, dict): images = [images]

            # 장소 명칭으로 폴더 생성
            # 폴더명에 사용할 수 없는 특수문자 제거
            safe_title = "".join([c for c in title if c.isalnum() or c in (' ', '_')]).strip()
            save_dir = os.path.join(base_path, safe_title)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)

            for idx, img in enumerate(images):
                img_url = img.get('originimgurl')
                if img_url:
                    # 이미지 다운로드 및 저장
                    img_res = requests.get(img_url, timeout=10)
                    file_path = os.path.join(save_dir, f"{contentId}_{idx}.jpg")
                    with open(file_path, 'wb') as f:
                        f.write(img_res.content)
                    print(f"   ㄴ [이미지 저장 완료] {idx+1}번 사진")
                    
    except Exception as e:
        print(f"   ㄴ 상세 이미지 처리 중 오류: {e}")

# --- 실행부 ---
# 부산의 거점별 수집 시작
# (해운대, 남포동, 서면 좌표)
points = [('129.1585', '35.1587'), ('129.0401', '35.1051'), ('129.0595', '35.1555')]

for mx, my in points:
    get_busan_data(mx, my)
    print("\n" + "="*50 + "\n거점 수집 완료. 다음 거점으로 이동합니다.")
    time.sleep(1)


🚀 좌표 (129.1585, 35.1587) 주변 관광지 검색을 시작합니다...
✅ 총 50개의 장소를 발견했습니다.
--------------------------------------------------
📍 현재 수집 중인 장소: [경상좌수영성지] (ID: 129631)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [고흐의 길] (ID: 3027738)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [광안리해변 테마거리] (ID: 127925)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [광안리해수욕장] (ID: 126078)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [달맞이동산] (ID: 2875837)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [대천공원] (ID: 2875849)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [더베이101] (ID: 2350092)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [동백공원] (ID: 128810)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [마린시티] (ID: 2617724)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [망미단길] (ID: 3033472)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [미포항] (ID: 2784330)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [민락수변공원] (ID: 126799)
   ㄴ 갤러리 이미지가 없습니다. 다음 장소로 이동합니다.
📍 현재 수집 중인 장소: [민락해변공원] (ID: 3